In [ ]:
!pip install diffusers

In [ ]:
# ref: https://huggingface.co/docs/diffusers/main/en/using-diffusers/inpaint#stable-diffusion-inpainting

import os
import platform

import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

In [ ]:
pipeline = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16, variant="fp16"
)

if not platform.system() == 'Linux':
    if torch.backends.mps.is_available():
        pipeline.enable_model_cpu_offload(device=torch.device("cpu"))
else:
    pipeline.enable_model_cpu_offload()
    pipeline.enable_xformers_memory_efficient_attention()

init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png")

if not platform.system() == 'Linux':
    if torch.backends.mps.is_available():
        generator = torch.Generator("cpu").manual_seed(92)
else:
    generator = torch.Generator("cuda").manual_seed(92)

prompt = "concept art digital painting of an elven castle, inspired by lord of the rings, highly detailed, 8k"
image = pipeline(
    prompt=prompt,
    image=init_image,
    mask_image=mask_image,
    generator=generator
).images[0]

result_img = make_image_grid([init_image, mask_image, image], rows=1, cols=3, resize=512)

result_img.save('demo_result.png')